# Part A - (1) Feature Extraction
The code has been modified to split the data into 3 sets: Training (70%), Calibration (15%) and Testing (15%) using sklearn's test_train_split function.

In [ ]:
import torch
import timm
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
from tqdm import tqdm
import pandas as pd
import os

# 1. Load pretrained Vision Transformer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit = timm.create_model("vit_base_patch16_224", pretrained=True).to(device)
vit.eval() # disable dropout, etc.

# 2. Define preprocessing consistent with ImageNet training
preprocess = transforms.Compose([
transforms.Resize(256),
transforms.CenterCrop(224),
transforms.ToTensor(),
transforms.Normalize(
mean=[0.485, 0.456, 0.406],
std=[0.229, 0.224, 0.225]
),
])

# 3. Directory containing CelebA images (modify path)
image_dir = "/home/arty/projects/cs342_dataset/img_align_celeba"
image_list = sorted(os.listdir(image_dir))[:20000] # sample subset

# Get labeled attributes from dataset
attr_file = "/home/arty/projects/cs342_dataset/list_attr_celeba.csv"
attr_df = pd.read_csv(attr_file)

# Split images into train (70%). calib (15%), test (15%)
train_set, remaining_set = train_test_split(
    image_list, 
    test_size=0.30, 
    random_state=42, # Use a fixed random_state for reproducibility
    shuffle=True
)
calib_set, test_set = train_test_split(
    remaining_set, 
    test_size=0.50, # 50% of the remaining 30% = 15% of the total
    random_state=42, # Using the same random_state is good practice
    shuffle=True
)

sets = [train_set, calib_set, test_set]

# 4. Extract embeddings
for split_name, image_list in zip(["train", "calib", "test"], sets):
    print(f"Processing {split_name} set with {len(image_list)} images...")

    all_features = []
    with torch.no_grad():
        for fname in tqdm(image_list):
            img = Image.open(os.path.join(image_dir, fname)).convert("RGB")
            x = preprocess(img).unsqueeze(0).to(device)
            features = vit.forward_features(x)[:, 0, :] # shape: (1, 768)
            all_features.append(features.cpu().numpy())
    all_features = np.concatenate(all_features, axis=0)

    # Add 'Smiling' class label as the last column
    smiling_labels = []
    for fname in image_list:
        index = int(fname.split('.')[0]) 
        smiling_label = 1 if attr_df.at[index, 'Smiling'] == 1 else 0
        smiling_labels.append(smiling_label)
    smiling_labels = np.array(smiling_labels).reshape(-1, 1)
    all_features = np.hstack((all_features, smiling_labels))

    print("Feature matrix shape:", all_features.shape) # e.g. (20000, 768 + 1)

    # 5. Save for later use
    np.save(f"celeba_vit_embeddings_{split_name}.npy", all_features)

# Part A - (2) Train Classifiers
Implement a Naive Bayes classifier to predict the Smiling attribute
(1 = smiling, 0 = not smiling) and report test accuracy. The Naive Bayes classifier should
be implemented by you and not be a call to a library function. The features here
are numerical (not categorical). So conditioned on the label, model each feature as an
independent Gaussian. Your algorithm should estimate the mean and variance of each of
these Gaussians, using the training data set

In [67]:
import pandas as pd
import numpy as np
from math import exp, sqrt, log, pi

train_data = pd.DataFrame(np.load("./celeba_vit_embeddings_train.npy"))
train_data.head()

num_features = train_data.shape[1] - 1

# First separate the data by class
pos_df = train_data[train_data[num_features]==1]
neg_df = train_data[train_data[num_features]==0]

# Now we calculate the mean and std deviation for each feature in both classes
pos_std_devs = pos_df.std()
pos_means = pos_df.mean()
neg_std_devs = neg_df.std()
neg_means = neg_df.mean()

# Calculate PDF for a given input feature vector
def calc_probs_vectorized(vector):
    total_rows = train_data.shape[0]
    
    # Prior probabilities
    log_pos_prior = np.log(pos_df.shape[0] / total_rows)
    log_neg_prior = np.log(neg_df.shape[0] / total_rows)
    
    # Vectorized Gaussian log probability calculation
    def gaussian_log_prob(vector, means, stddevs):
        # Add small epsilon to avoid zero stddev
        stddevs = np.where(stddevs == 0, 1e-9, stddevs)
        return -0.5 * np.log(2 * pi) - np.log(stddevs) - ((vector - means) ** 2) / (2 * stddevs ** 2)
    
    # Calculate log probabilities for all features
    pos_log_probs = gaussian_log_prob(vector[:num_features], pos_means[:num_features], pos_std_devs[:num_features])
    neg_log_probs = gaussian_log_prob(vector[:num_features], neg_means[:num_features], neg_std_devs[:num_features])
    
    # Sum all feature log probabilities and add priors
    log_pos_prob = log_pos_prior + np.sum(pos_log_probs)
    log_neg_prob = log_neg_prior + np.sum(neg_log_probs)
    
    return log_pos_prob, log_neg_prob
    
# Run predictions on the training set
print(calc_probs(list(pos_df.iloc[1])))

(np.float64(-1475.2638435704364), np.float64(-1486.0390792175162))
